### **1. Fuente de información**

In [1]:
# 1. Fuente de información (leyendo un archivo de texto)

def leer_archivo(nombre_archivo):
    try:
        with open(nombre_archivo, 'r') as archivo:
            contenido = archivo.read()
            return contenido
    except FileNotFoundError:
        print(f"El archivo '{nombre_archivo}' no fue encontrado.")
        return ""

### **2. Transmisor**

In [2]:
# Paso 1: Convertir el mensaje de texto a una cadena binaria
def texto_a_binario(mensaje):
    binario = ''.join(format(ord(char), '08b') for char in mensaje)
    return binario

In [3]:
# Paso 2: Dividir la cadena binaria en bytes (8 bits)
def dividir_cadena(binario):
    return [binario[i:i+8] for i in range(0, len(binario), 8)]

In [4]:
# Paso 3: Calcular las frecuencias de cada byte
from collections import defaultdict

def calcular_frecuencias(bytes):
    frecuencias = defaultdict(int)
    for byte in bytes:
        frecuencias[byte] += 1
    return frecuencias

In [5]:
# Paso 4: Construir la tabla de codificación Shannon-Fano
def construir_tabla(frecuencias):
    # Ordenar las frecuencias en orden descendente
    frecuencias_ordenadas = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)

    # Crear la tabla de codificación Shannon-Fano
    tabla_codificacion = {}
    def asignar_codigos(frecuencias_ordenadas, codigo=''):
        if len(frecuencias_ordenadas) == 1:
            tabla_codificacion[frecuencias_ordenadas[0][0]] = codigo
        else:
            split_point = len(frecuencias_ordenadas) // 2
            for byte, frecuencia in frecuencias_ordenadas[:split_point]:
                tabla_codificacion[byte] = codigo + '0'
            for byte, frecuencia in frecuencias_ordenadas[split_point:]:
                tabla_codificacion[byte] = codigo + '1'
            asignar_codigos(frecuencias_ordenadas[:split_point], codigo + '0')
            asignar_codigos(frecuencias_ordenadas[split_point:], codigo + '1')

    asignar_codigos(frecuencias_ordenadas)
    return tabla_codificacion

In [6]:
# Paso 5: Codificar el mensaje utilizando la tabla de codificación Shannon-Fano
def codificacion_shannon(bytes_binarios, tabla_codificacion):
    texto_codificado = ''.join(tabla_codificacion[byte] for byte in bytes_binarios)
    return texto_codificado

### **3. Canal**

In [7]:
import random

# Paso 1: Dividir el mensaje codificado en paquetes
def dividir_en_paquetes(mensaje_codificado, tamano_paquete):
    paquetes = [mensaje_codificado[i:i + tamano_paquete] for i in range(0, len(mensaje_codificado), tamano_paquete)]
    return paquetes

In [8]:
# Función para simular la pérdida de paquetes
def perdida_de_paquetes(paquetes, probabilidad_perdida):
    paquetes_perdidos = []
    for paquete in paquetes:
        if random.random() < probabilidad_perdida:
            paquetes_perdidos.append(None)
            print(f"Paquete {paquete} se perdió. Retransmitiendo...")
        else:
            paquetes_perdidos.append(paquete)
    return paquetes_perdidos

In [9]:
# Función para recibir los paquetes y solicitar reenvío de los perdidos
def recibir_y_reenviar(paquetes_perdidos, mensaje_codificado):
    nuevo_mensaje = []

    for paquete in paquetes_perdidos:
        if paquete is None:
            # Si el paquete se perdió, solicitar reenvío
            indice = len(nuevo_mensaje)
            while paquete is None:
                paquete = mensaje_codificado[indice]  # Simula el reenvío
                nuevo_mensaje.append(paquete)
                indice += 1
        else:
            nuevo_mensaje.append(paquete)

    return nuevo_mensaje

### **4. Receptor**

In [10]:
# Paso 6: Decodificar el texto codificado utilizando la tabla de codificación Shannon-Fano
def decodificacion_shannon(texto_codificado, tabla_codificacion):
    texto_decodificado = ''
    codigo_actual = ''
    for bit in texto_codificado:
        codigo_actual += bit
        if codigo_actual in tabla_codificacion.values():
            byte = [key for key, value in tabla_codificacion.items() if value == codigo_actual][0]
            texto_decodificado += byte
            codigo_actual = ''
    return texto_decodificado

# Función para convertir una cadena binaria en texto
def binario_a_texto(binario):
    texto = ""
    for i in range(0, len(binario), 8):
        byte = binario[i:i+8]
        char = chr(int(byte, 2))
        texto += char
    return texto


### **5. Destino de información**

In [11]:
nombre_archivo_fuente = "fuente.txt"

texto_original = leer_archivo(nombre_archivo_fuente)

print("Mensaje:", texto_original)
print()

mensaje_binario = texto_a_binario(texto_original)
print("Mensaje binario:",mensaje_binario)

bytes_binarios = dividir_cadena(mensaje_binario)
print("Bytes binarios:",bytes_binarios)

frecuencias = calcular_frecuencias(bytes_binarios)
# print("Frecuencias:",frecuencias)

tabla_codificacion_sf = construir_tabla(frecuencias)
mensaje_codificado = codificacion_shannon(bytes_binarios, tabla_codificacion_sf)
print("Mensaje codificado", mensaje_codificado)

tamano_paquete = 8  # Define el tamaño de cada paquete
paquetes = dividir_en_paquetes(mensaje_codificado, tamano_paquete)
print("Paquetes: ", paquetes)
print()

probabilidad_perdida = 0.4  # Probabilidad de pérdida de paquetes
paquetes_perdidos = perdida_de_paquetes(paquetes, probabilidad_perdida)
print()
print("Paquetes perdidos:",paquetes_perdidos)
print()

nuevo_mensaje = recibir_y_reenviar(paquetes_perdidos, paquetes)
print("Nuevo mensaje recuperado:",nuevo_mensaje)

mensaje_decodificado = decodificacion_shannon(mensaje_codificado, tabla_codificacion_sf)
print("Mensaje decodificado:", mensaje_decodificado)

# Paso 6: Decodificar el mensaje recibido
nuevo_mensaje_decodificado = decodificacion_shannon(''.join(nuevo_mensaje), tabla_codificacion_sf)
print("Nuevo mensaje decodificado:",nuevo_mensaje_decodificado)

mensaje_binario_decodificado = nuevo_mensaje_decodificado
mensaje_texto_decodificado = binario_a_texto(mensaje_binario_decodificado)

print()
print("Mensaje recivido:",mensaje_texto_decodificado)
# Comparar el mensaje decodificado con el texto original
if mensaje_texto_decodificado == texto_original:
    print()
    print(" <-- El mensaje decodificado coincide con el texto original -->")
else:
    print()
    print("--> El mensaje decodificado no coincide con el texto original <--")

Mensaje: Hola, esto es un mensaje de prueba para la comunicacion.

Mensaje binario: 0100100001101111011011000110000100101100001000000110010101110011011101000110111100100000011001010111001100100000011101010110111000100000011011010110010101101110011100110110000101101010011001010010000001100100011001010010000001110000011100100111010101100101011000100110000100100000011100000110000101110010011000010010000001101100011000010010000001100011011011110110110101110101011011100110100101100011011000010110001101101001011011110110111000101110
Bytes binarios: ['01001000', '01101111', '01101100', '01100001', '00101100', '00100000', '01100101', '01110011', '01110100', '01101111', '00100000', '01100101', '01110011', '00100000', '01110101', '01101110', '00100000', '01101101', '01100101', '01101110', '01110011', '01100001', '01101010', '01100101', '00100000', '01100100', '01100101', '00100000', '01110000', '01110010', '01110101', '01100101', '01100010', '01100001', '00100000', '01110000', '01100001', '01110